#SETUP

In [60]:
BATCH_SIZE=80
MAX_SEQ_LENGTH=32
MODEL_NAME='google-t5/t5-small'
NUM_EPOCHS=20

In [61]:
!pip install datasets
!pip install evaluate
!pip install sacrebleu
!pip install sacremoses
!pip install wandb
!pip install polars
!pip install transformers
!pip install rouge_score
!pip install sentencepiece
!pip install scikit-learn
!pip install accelerate


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Looking in indexes: https://pypi.org/simple, http://localhost:8081


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Looking in indexes: https://pypi.org/simple, http://localhost:8081


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


...

Looking in indexes: https://pypi.org/simple, http://localhost:8081


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Looking in indexes: https://pypi.org/simple, http://localhost:8081


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Looking in indexes: https://pypi.org/simple, http://localhost:8081


In [62]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from datasets import Dataset
import os
import json
import urllib.request

import wandb
wandb.init(mode="disabled")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


#DATA PIPELINE

##Raw dataload

In [63]:
DE_TEST_FILE  = 'deu_test.json'
DE_TRAIN_FILE = 'deu_training.json'
FR_TEST_FILE  = 'fra_test.json'
FR_TRAIN_FILE = 'fra_training.json'
ES_TEST_FILE  = 'spa_test.json'
ES_TRAIN_FILE = 'spa_training.json'

def read_json_from_git(file_path):
    url = f'https://raw.githubusercontent.com/wortcook/LLMs-from-scratch/refs/heads/main/data/{file_path}'

    text_json = None

    if not os.path.exists(file_path):
        with urllib.request.urlopen(url) as response:
            text_data = response.read().decode('utf-8')
        with open(file_path, "w", encoding="utf-8") as file:
            file.write(text_data)
            text_json = json.loads(text_data)
    else:
        with open(file_path, "r", encoding="utf-8") as file:
          text_json = json.load(file)

    return text_json


DE_TEST  = read_json_from_git(DE_TEST_FILE)
DE_TRAIN = read_json_from_git(DE_TRAIN_FILE)
FR_TEST  = read_json_from_git(FR_TEST_FILE)
FR_TRAIN = read_json_from_git(FR_TRAIN_FILE)
ES_TEST  = read_json_from_git(ES_TEST_FILE)
ES_TRAIN = read_json_from_git(ES_TEST_FILE)


##Data Extraction

In [64]:
import polars as pl

def create_translation_df(data, src_lang, trg_lang, src_instruction_lang, target_instruction_lang, is_reversed=False):
  """Creates a Polars DataFrame from a list of JSON data.

  Args:
    data: A list of JSON objects, each containing 'source' and 'target' keys.
    src_lang: The source language code (e.g., 'en').
    trg_lang: The target language code (e.g., 'fr').

  Returns:
    A Polars DataFrame with columns 'src_lang', 'trg_lang', 'source', and 'target'.
  """

  the_source = "source"
  the_target = "target"

  if is_reversed:
    # Reverse the source and target languages
    src_lang, trg_lang = trg_lang, src_lang
    the_source, the_target = the_target, the_source
    src_instruction_lang, target_instruction_lang = target_instruction_lang, src_instruction_lang

  # 1. Extract source and target sentences from the JSON data.
  source_sentences = [f"translate {src_instruction_lang} to {target_instruction_lang}: {item[the_source]}" for item in data]
  target_sentences = [item[the_target] for item in data]

  # 2. Create a Polars DataFrame.
  df = pl.DataFrame({
      'src_lang': [src_lang] * len(data),  # Repeat src_lang for all rows
      'trg_lang': [trg_lang] * len(data),  # Repeat trg_lang for all rows
      'source': source_sentences,
      'target': target_sentences,
  })

  return df

en_fr_df = create_translation_df(FR_TRAIN, 'en', 'fr', 'English', 'French')
en_es_df = create_translation_df(ES_TRAIN, 'en', 'es', 'English', 'Spanish')
en_de_df = create_translation_df(DE_TRAIN, 'en', 'de', 'English', 'German')

print(en_fr_df.head())
print(en_es_df.head())
print(en_de_df.head())

shape: (5, 4)
┌──────────┬──────────┬─────────────────────────────────┬─────────────────────────────────┐
│ src_lang ┆ trg_lang ┆ source                          ┆ target                          │
│ ---      ┆ ---      ┆ ---                             ┆ ---                             │
│ str      ┆ str      ┆ str                             ┆ str                             │
╞══════════╪══════════╪═════════════════════════════════╪═════════════════════════════════╡
│ en       ┆ fr       ┆ translate English to French: I… ┆ Il pleut beaucoup maintenant.   │
│ en       ┆ fr       ┆ translate English to French: P… ┆ Veuillez vous reporter au guid… │
│ en       ┆ fr       ┆ translate English to French: T… ┆ Tom n'a pas l'intention de qui… │
│ en       ┆ fr       ┆ translate English to French: N… ┆ Personne ne t'aime.             │
│ en       ┆ fr       ┆ translate English to French: D… ┆ Ne lui dis pas.                 │
└──────────┴──────────┴─────────────────────────────────┴─────────

##Data Setup

After this next cell, we no longer worry about which languages.

In [65]:
total_df = pl.concat([en_fr_df, en_de_df])
print(total_df.head())

#Find the maximum character length for source and target
max_source_length = total_df['source'].str.len_chars().max()
max_target_length = total_df['target'].str.len_chars().max()

max_length = max(max_source_length, max_target_length)

print(f"Maximum source length: {max_source_length}")
print(f"Maximum target length: {max_target_length}")
print(f"Maximum length: {max_length}")

#create train-validation splits
from sklearn.model_selection import train_test_split

fr_trn, fr_val = train_test_split(en_fr_df, test_size=0.1, random_state=42)
es_trn, es_val = train_test_split(en_es_df, test_size=0.1, random_state=42)
de_trn, de_val = train_test_split(en_de_df, test_size=0.1, random_state=42)

train_df = pl.concat([fr_trn, de_trn])
val_df = pl.concat([fr_val, de_val])

print(train_df.shape)
print(val_df.shape)

print(train_df.head())

en_fr_test = create_translation_df(FR_TEST, 'en', 'fr', 'English', 'French')
en_es_test = create_translation_df(ES_TEST, 'en', 'es', 'English', 'Spanish')
en_de_test = create_translation_df(DE_TEST, 'en', 'de', 'English', 'German')

#convert to input and output arrays
test_df = pl.concat([en_fr_test, en_de_test])

shape: (5, 4)
┌──────────┬──────────┬─────────────────────────────────┬─────────────────────────────────┐
│ src_lang ┆ trg_lang ┆ source                          ┆ target                          │
│ ---      ┆ ---      ┆ ---                             ┆ ---                             │
│ str      ┆ str      ┆ str                             ┆ str                             │
╞══════════╪══════════╪═════════════════════════════════╪═════════════════════════════════╡
│ en       ┆ fr       ┆ translate English to French: I… ┆ Il pleut beaucoup maintenant.   │
│ en       ┆ fr       ┆ translate English to French: P… ┆ Veuillez vous reporter au guid… │
│ en       ┆ fr       ┆ translate English to French: T… ┆ Tom n'a pas l'intention de qui… │
│ en       ┆ fr       ┆ translate English to French: N… ┆ Personne ne t'aime.             │
│ en       ┆ fr       ┆ translate English to French: D… ┆ Ne lui dis pas.                 │
└──────────┴──────────┴─────────────────────────────────┴─────────

###Tokenize and cache locally for re-runs

In [66]:
from transformers import AutoTokenizer

# Create a Tokenizer for Machine Translation
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

In [67]:
#check if the pickled encodings exists
import os
import pickle

def check_pickled_encodings(prefix, train_df, val_df):
  if os.path.exists(f'{prefix}_input_encoding.pkl'):
    with open(f'{prefix}_input_encoding.pkl', 'rb') as f:
      input_encoding = pickle.load(f)

    with open(f'{prefix}_label_encoding.pkl', 'rb') as f:
      label_encoding = pickle.load(f)

    return input_encoding, label_encoding

  else:
    input_encoding = tokenizer(train_df['source'].to_list(), return_tensors='pt', padding='max_length', max_length=MAX_SEQ_LENGTH, truncation=True)
    label_encoding = tokenizer(train_df['target'].to_list(), return_tensors='pt', padding='max_length', max_length=MAX_SEQ_LENGTH, truncation=True)

    with open(f'{prefix}_input_encoding.pkl', 'wb') as f:
      pickle.dump(input_encoding, f)

    with open(f'{prefix}_label_encoding.pkl', 'wb') as f:
      pickle.dump(label_encoding, f)

    return input_encoding, label_encoding

In [68]:
train_input_encoding, train_label_encoding = check_pickled_encodings('train', train_df, val_df)
val_input_encoding, val_label_encoding = check_pickled_encodings('val', train_df, val_df)

#define dataframe as input_ids, attention_mask, and labels
train_dataset = Dataset.from_dict({'input_ids': train_input_encoding['input_ids'], 'attention_mask': train_input_encoding['attention_mask'], 'labels': train_label_encoding['input_ids']})
val_dataset = Dataset.from_dict({'input_ids': val_input_encoding['input_ids'], 'attention_mask': val_input_encoding['attention_mask'], 'labels': val_label_encoding['input_ids']})

##Sample Data Setup

In [69]:
sampled = test_df.sample(50)

samples_to_translate = sampled['source'].to_list();
samples_target = sampled['target'].to_list()

#Execution

##Model Setup

In [70]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq

# Instantiate a Seq2Seq model from the specified pretrained model
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME)

# Define a data collator for Seq2Seq tasks
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

# 'model' is now an instance of the Seq2Seq model, and 'data_collator' is set up for data preparation.


In [71]:
import evaluate

metric = evaluate.load("sacrebleu")

In [72]:
'''
This custom compute_metrics function calculates the BERTScore metric for evaluating text generation quality.
It processes the predictions and labels, decodes them using the tokenizer, and computes the BERTScore using the bert_metric.

Input:
preds_and_labels: A tuple containing predictions and labels.
preds: Predictions from the model.
labels: Ground truth labels.
Output:
A dictionary containing the computed BERTScore.
'''

model_save_tracker = 0
trainer = None

def compute_metrics(preds_and_labels):

    preds, labels = preds_and_labels

    # Decode the predictions and labels using the tokenizer, skipping special tokens

    # preds = np.where(preds != -100, preds, tokenizer.pad_token_id)

    # decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100 (masked tokens) in labels with the pad token ID
    # labels = np.where(labels != -100, labels, tokenizer.pad_token_id)

    # Decode the labels using the tokenizer, skipping special tokens
    # decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Compute BERTScore using decoded predictions and labels
    # result = metric.compute(predictions=decoded_preds, references=decoded_labels)



    global model_save_tracker
    global trainer

    model_save_tracker += 1

    #save the model with padded model_save_tracker
    trainer.save_model(f"{model_save_tracker:03}")

    # Return the BERTScore as a dictionary
    return {'Score': np.mean(result['score'])}

##Trainer

In [73]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer

'''
This code snippet configures training arguments and sets up a Seq2Seq trainer for model.
It prepares the training environment for fine-tuning Seq2Seq model on the specified datasets.

Dependencies:
Seq2SeqTrainingArguments: This code uses the Seq2SeqTrainingArguments class to specify training arguments.
Seq2SeqTrainer: The Seq2SeqTrainer class is used to set up the training process.
Input (Training Arguments):
output_dir='finetuned': The directory where the fine-tuned model and logs will be saved.
per_device_train_batch_size=16: Batch size for training on each device (e.g., GPU).
per_device_eval_batch_size=20: Batch size for evaluation on each device.
predict_with_generate=True: Indicates that the model should generate predictions during evaluation.
evaluation_strategy='epoch': Evaluation strategy (in this case, evaluation is performed after each epoch).
weight_decay=0.01: Weight decay for regularization.
save_total_limit=3: Limit on the total number of checkpoints to save.
learning_rate=2e-5: Learning rate for fine-tuning.
Output:
trainer: An instance of the Seq2Seq trainer, which is set up for training and evaluation.
'''

# Define training arguments for Seq2Seq model
training_args = Seq2SeqTrainingArguments(
    output_dir='finetuned',
    per_device_train_batch_size=256,
    per_device_eval_batch_size=20,
    predict_with_generate=True,
    eval_strategy='epoch',
    weight_decay=0.01,
    save_total_limit=3,
    learning_rate=2e-5,
    num_train_epochs=NUM_EPOCHS,
)

# Create a Seq2Seq trainer with the specified arguments
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator,
    processing_class=tokenizer,
    compute_metrics=compute_metrics,
)

# 'trainer' is now set up for fine-tuning Seq2Seq model.

##Translation Functions

In [74]:
def translate_batch(texts):
  """Translates a batch of texts using the pre-trained model and tokenizer.

  Args:
    texts: A list of strings to be translated.

  Returns:
    A list of translated strings.
  """
  # 1. Tokenize the input texts
  inputs = tokenizer(texts, return_tensors='pt', padding='max_length', max_length=MAX_SEQ_LENGTH, truncation=True).to(model.device)

  # 2. Pass the tokenized inputs to the model
  outputs = model.generate(**inputs)

  # 3. Decode the model's output to get the translated texts
  translated_texts = tokenizer.batch_decode(outputs, skip_special_tokens=True)

  return translated_texts

##Sample Pre-fine tuning

In [75]:
test_preds = translate_batch(samples_to_translate)

print(json.dumps(test_preds, indent=4))

#Score our translations
print(json.dumps(metric.compute(predictions=test_preds, references=samples_target), indent=4))

[
    "La plupart des \u00e9l\u00e8ves se rendent \u00e0 l'\u00e9cole.",
    "Tom ist seit einer Woche weg.",
    "Tom ist wirklich gut am Schwimmen.",
    "Vous \u00eates trop critiques \u00e0 l'\u00e9gard des lacunes d",
    "Tom hat eine j\u00fcngere Schwester.",
    "Tom hielt ein Glas Whisky in der einen Hand und eine Zigarette in der anderen",
    "Cela aurait \u00e9t\u00e9 embarrassant.",
    "Ich mu\u00df nicht denken.",
    "Y a-t-il quelqu'un qui a un kleenex?",
    "L'eau \u00e9tait chaude.",
    "Wir trafen uns am Flughafen.",
    "Ich bin in einer Schwanz fertig.",
    "Chacun a apport\u00e9 des contributions.",
    "Ich mag nicht lachen.",
    "Tom savait qu'il avait \u00e9t\u00e9 tu\u00e9.",
    "Je me demande quel sera le temps demain.",
    "Tom war f\u00fcr W\u00e4rme gekleidet und nicht f\u00fcr Mode.",
    "Haben Sie sich entschieden, nach Japan zu gehen?",
    "V\u00f6gel sangen im Wald.",
    "Je voudrais vivre pr\u00e8s de votre maison.",
    "Tom entschuldigte s

In [76]:
# Assuming to_translate and test_preds are lists
joined_df = pd.DataFrame({'Original': samples_to_translate, 'Target': samples_target, 'Translation': test_preds})

# Display or use the joined_df DataFrame
joined_df.head(50)

,Original,Target,Translation
0,translate English to French: Most students wal...,La plupart des élèves vont à l'école à pied.,La plupart des élèves se rendent à l'école.
1,translate English to German: Tom has been gone...,Tom ist schon eine ganze Woche weg.,Tom ist seit einer Woche weg.
2,translate English to German: Tom is really goo...,Tom kann sehr gut schwimmen.,Tom ist wirklich gut am Schwimmen.
3,translate English to French: You're too critic...,Tu es trop critique des défauts des autres.,Vous êtes trop critiques à l'égard des lacunes d
4,translate English to German: Tom has a younger...,Tom hat eine jüngere Schwester.,Tom hat eine jüngere Schwester.
5,translate English to German: Tom was holding a...,Tom hielt in der einen Hand ein Glas Whiskey u...,Tom hielt ein Glas Whisky in der einen Hand un...
6,translate English to French: That would've bee...,C'eût été embarrassant.,Cela aurait été embarrassant.
7,translate English to German: I don't care.,Egal.,Ich muß nicht denken.
8,translate English to French: Does anybody have...,Est-ce que quelqu'un a un mouchoir ?,Y a-t-il quelqu'un qui a un kleenex?
9,translate English to French: The water was hot.,L'eau était chaude.,L'eau était chaude.


####Pre-training metrics

In [77]:
def compute_test_metrics(df_to_test):
  inp_encoding = tokenizer(df_to_test['source'].to_list(), return_tensors='pt', padding='max_length', max_length=MAX_SEQ_LENGTH, truncation=True)
  lbl_encoding = tokenizer(df_to_test['target'].to_list(), return_tensors='pt', padding='max_length', max_length=MAX_SEQ_LENGTH, truncation=True)

  test_ds = Dataset.from_dict({'input_ids': inp_encoding['input_ids'], 'attention_mask': inp_encoding['attention_mask'], 'labels': lbl_encoding['input_ids']})

  return trainer.evaluate(test_ds)

In [78]:
































#print(json.dumps(compute_test_metrics(en_fr_test), indent=4))
#print(json.dumps(compute_test_metrics(en_de_test), indent=4))


##Training

In [79]:
trainer.save_model('my_model')

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss


Save after training

In [ ]:
trainer.save_model('my_model')

## Testing - Sample post fine-tuning

In [ ]:
test_preds = translate_batch(samples_to_translate)

print(json.dumps(test_preds, indent=4))

#Score our translations
print(json.dumps(metric.compute(predictions=test_preds, references=samples_target), indent=4))

[
    "Je ne suis pas s\u00fbre de pouvoir en faire.",
    "Sind Sie verheiratet oder allein?",
    "Hast du den Filter ge\u00e4ndert?",
    "Ich habe alles versucht.",
    "Laissez-nous franchement envers nous.",
    "Das klingt kompliziert.",
    "Ich habe Milch getrunken.",
    "Ich dachte, du liebst Tom.",
    "Es w\u00e4re nicht davon abh\u00e4ngig gewesen.",
    "Il ne me comprend pas.",
    "Tom ist wirklich betrunken, nicht wahr?",
    "Laissez-moi le porter pour toi.",
    "Comment t'entendre?",
    "Sie werden einen Job finden.",
    "Wir wissen, wer Sie sind.",
    "Sie wurde vom Club ausgeschlossen.",
    "Qu'est-ce que l'affaire vous occupe?",
    "Das ist das Buch, das Tom mir gegeben hat.",
    "W\u00e4hlen Sie das Richtige, nicht das, was leicht ist.",
    "Hat sie gut geschlafen?",
    "Ich hatte nicht genug Zeit, alles zu essen, was serviert wurde.",
    "Magst du, wie du ausschaust?",
    "Er hatte die richtige Idee.",
    "Mein Vater mag Pizza sehr gerne.",
    "Tu 

In [ ]:
# Assuming to_translate and test_preds are lists
joined_df = pd.DataFrame({'Original': samples_to_translate, 'Target': samples_target, 'Translation': test_preds})

# Display or use the joined_df DataFrame
joined_df.head(50)

,Original,Target,Translation
0,translate English to French: I'm not sure that...,Je ne suis pas certain d'être prêt pour cela.,Je ne suis pas sûre de pouvoir en faire.
1,translate English to German: Are you married o...,Bist du verheiratet oder alleinstehend?,Sind Sie verheiratet oder allein?
2,translate English to German: Did you change th...,Habt ihr den Filter getauscht?,Hast du den Filter geändert?
3,translate English to German: I tried everything.,Ich habe alles versucht.,Ich habe alles versucht.
4,translate English to French: Let's be frank wi...,Soyons francs les uns envers les autres.,Laissez-nous franchement envers nous.
5,translate English to German: That sounds compl...,Das hört sich kompliziert an.,Das klingt kompliziert.
6,translate English to German: I drank milk.,Ich trank Milch.,Ich habe Milch getrunken.
7,translate English to German: I thought you lov...,"Ich dachte, du liebst Tom.","Ich dachte, du liebst Tom."
8,translate English to German: It wouldn't have ...,Es hätte nichts gemacht.,Es wäre nicht davon abhängig gewesen.
9,translate English to French: He doesn't unders...,Il ne me comprend pas.,Il ne me comprend pas.


#Get Final Metrics

In [ ]:
!zip -r "mymodel.zip" "my_model"

  adding: my_model/ (stored 0%)
  adding: my_model/spiece.model (deflated 48%)
  adding: my_model/tokenizer.json (deflated 74%)
  adding: my_model/special_tokens_map.json (deflated 85%)
  adding: my_model/generation_config.json (deflated 29%)
  adding: my_model/model.safetensors (deflated 8%)
  adding: my_model/training_args.bin (deflated 52%)
  adding: my_model/tokenizer_config.json (deflated 95%)
  adding: my_model/config.json (deflated 62%)


###French Metrics

In [ ]:
print(json.dumps(compute_test_metrics(en_fr_test), indent=4))


{
    "eval_loss": 0.23714298009872437,
    "eval_model_preparation_time": 0.0029,
    "eval_Score": 46.71735847992946,
    "eval_runtime": 570.6458,
    "eval_samples_per_second": 81.571,
    "eval_steps_per_second": 4.08,
    "epoch": 20.0
}


###German Metrics

In [ ]:
print(json.dumps(compute_test_metrics(en_de_test), indent=4))


{
    "eval_loss": 0.268144428730011,
    "eval_model_preparation_time": 0.0029,
    "eval_Score": 44.151300158880005,
    "eval_runtime": 657.9979,
    "eval_samples_per_second": 84.467,
    "eval_steps_per_second": 4.223,
    "epoch": 20.0
}


###Total Metrics

In [ ]:
print(json.dumps(compute_test_metrics(test_df), indent=4))

{
    "eval_loss": 0.25402265787124634,
    "eval_model_preparation_time": 0.0029,
    "eval_Score": 45.31149527174555,
    "eval_runtime": 1231.2637,
    "eval_samples_per_second": 82.945,
    "eval_steps_per_second": 4.148,
    "epoch": 20.0
}


#Adding Spanish

##Create new combined dataset

In [ ]:
full_train_df = pl.concat([fr_trn, de_trn, es_trn])
full_val_df = pl.concat([fr_val, de_val, es_trn])

print(full_train_df.shape)
print(full_val_df.shape)

print(full_train_df.head())

en_fr_test = create_translation_df(FR_TEST, 'en', 'fr', 'English', 'French')
en_es_test = create_translation_df(ES_TEST, 'en', 'es', 'English', 'Spanish')
en_de_test = create_translation_df(DE_TEST, 'en', 'de', 'English', 'German')

#convert to input and output arrays
test_df = pl.concat([en_fr_test, en_de_test, en_es_test])

full_train_input_encoding, full_train_label_encoding = check_pickled_encodings('full_train', full_train_df, full_val_df)
full_val_input_encoding, full_val_label_encoding = check_pickled_encodings('full_val', full_train_df, full_val_df)

#define dataframe as input_ids, attention_mask, and labels
full_train_dataset = Dataset.from_dict({'input_ids': full_train_input_encoding['input_ids'], 'attention_mask': full_train_input_encoding['attention_mask'], 'labels': full_train_label_encoding['input_ids']})
full_val_dataset = Dataset.from_dict({'input_ids': full_val_input_encoding['input_ids'], 'attention_mask': full_val_input_encoding['attention_mask'], 'labels': full_val_label_encoding['input_ids']})

(393127, 4)
(66329, 4)
shape: (5, 4)
┌──────────┬──────────┬─────────────────────────────────┬─────────────────────────────────┐
│ src_lang ┆ trg_lang ┆ source                          ┆ target                          │
│ ---      ┆ ---      ┆ ---                             ┆ ---                             │
│ str      ┆ str      ┆ str                             ┆ str                             │
╞══════════╪══════════╪═════════════════════════════════╪═════════════════════════════════╡
│ en       ┆ fr       ┆ translate English to French: I… ┆ Je ne suis pas surprise que To… │
│ en       ┆ fr       ┆ translate English to French: T… ┆ Il y a des cas où l'honnêteté … │
│ en       ┆ fr       ┆ translate English to French: T… ┆ Tom ne veut qu'une seule tasse… │
│ en       ┆ fr       ┆ translate English to French: I… ┆ C'est un miracle que personne … │
│ en       ┆ fr       ┆ translate English to French: I… ┆ Je ne parviens à penser à rien… │
└──────────┴──────────┴────────────────────

In [ ]:
# Create a Seq2Seq trainer with the specified arguments
trainerEs = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=full_train_dataset,
    eval_dataset=full_val_dataset,
    data_collator=data_collator,
    processing_class=tokenizer,
    compute_metrics=compute_metrics,
)

print("Spanish")
print(json.dumps(compute_test_metrics(en_es_test), indent=4))

trainerEs.train()

Spanish


{
    "eval_loss": 2.185657262802124,
    "eval_model_preparation_time": 0.0029,
    "eval_Score": 0.1545866228891371,
    "eval_runtime": 338.8213,
    "eval_samples_per_second": 83.551,
    "eval_steps_per_second": 4.179,
    "epoch": 20.0
}


Step,Training Loss
500,0.392000
1000,0.380600
1500,0.373500
2000,0.372700
2500,0.368700
3000,0.365900
3500,0.364500
4000,0.361200
4500,0.359600
5000,0.358200


TrainOutput(global_step=30720, training_loss=0.34016752739747363, metrics={'train_runtime': 7755.0954, 'train_samples_per_second': 1013.855, 'train_steps_per_second': 3.961, 'total_flos': 6.650814548410368e+16, 'train_loss': 0.34016752739747363, 'epoch': 20.0})

In [ ]:
print("French")
print(json.dumps(compute_test_metrics(en_fr_test), indent=4))

print("German")
print(json.dumps(compute_test_metrics(en_de_test), indent=4))

print("Spanish")
print(json.dumps(compute_test_metrics(en_es_test), indent=4))


French


{
    "eval_loss": 0.21991431713104248,
    "eval_model_preparation_time": 0.0029,
    "eval_Score": 48.44710043605678,
    "eval_runtime": 574.3154,
    "eval_samples_per_second": 81.05,
    "eval_steps_per_second": 4.054,
    "epoch": 20.0
}
German
{
    "eval_loss": 0.2518235743045807,
    "eval_model_preparation_time": 0.0029,
    "eval_Score": 45.801654084631565,
    "eval_runtime": 661.7923,
    "eval_samples_per_second": 83.983,
    "eval_steps_per_second": 4.199,
    "epoch": 20.0
}
Spanish
{
    "eval_loss": 0.8703290224075317,
    "eval_model_preparation_time": 0.0029,
    "eval_Score": 10.012045442042199,
    "eval_runtime": 352.6496,
    "eval_samples_per_second": 80.275,
    "eval_steps_per_second": 4.015,
    "epoch": 20.0
}


In [ ]:


# Create a Seq2Seq trainer with the specified arguments
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=full_train_dataset,
    eval_dataset=full_val_dataset,
    data_collator=data_collator,
    processing_class=tokenizer,
    compute_metrics=compute_metrics,
)

trainer.train()

print("French")
print(json.dumps(compute_test_metrics(en_fr_test), indent=4))

print("German")
print(json.dumps(compute_test_metrics(en_de_test), indent=4))

print("Spanish")
print(json.dumps(compute_test_metrics(en_es_test), indent=4))

Step,Training Loss
500,0.322300
1000,0.321700
1500,0.318800
2000,0.319800
2500,0.318300
3000,0.317200
3500,0.316700
4000,0.314700
4500,0.314500
5000,0.314200


French


{
    "eval_loss": 0.2095681130886078,
    "eval_Score": 49.37181570217624,
    "eval_runtime": 579.9089,
    "eval_samples_per_second": 80.268,
    "eval_steps_per_second": 4.014,
    "epoch": 20.0
}
German
{
    "eval_loss": 0.24186338484287262,
    "eval_Score": 46.692165448193975,
    "eval_runtime": 670.1327,
    "eval_samples_per_second": 82.937,
    "eval_steps_per_second": 4.147,
    "epoch": 20.0
}
Spanish
{
    "eval_loss": 0.699350893497467,
    "eval_Score": 17.301287081023297,
    "eval_runtime": 352.9172,
    "eval_samples_per_second": 80.214,
    "eval_steps_per_second": 4.012,
    "epoch": 20.0
}


In [ ]:

# Create a Seq2Seq trainer with the specified arguments
for i in range(10):
  trainer = Seq2SeqTrainer(
      model=model,
      args=training_args,
      train_dataset=full_train_dataset,
      eval_dataset=full_val_dataset,
      data_collator=data_collator,
      processing_class=tokenizer,
      compute_metrics=compute_metrics,
  )

  trainer.train()

  print("French")
  print(json.dumps(compute_test_metrics(en_fr_test), indent=4))

  print("German")
  print(json.dumps(compute_test_metrics(en_de_test), indent=4))

  print("Spanish")
  print(json.dumps(compute_test_metrics(en_es_test), indent=4))

  trainer.save_model('my_model')

Step,Training Loss
500,0.292900
1000,0.293100
1500,0.290800
2000,0.291800
2500,0.291000
3000,0.290400
3500,0.289900
4000,0.288400
4500,0.288600
5000,0.288600


French


{
    "eval_loss": 0.202390655875206,
    "eval_Score": 50.113189242555784,
    "eval_runtime": 576.2765,
    "eval_samples_per_second": 80.774,
    "eval_steps_per_second": 4.04,
    "epoch": 20.0
}
German
{
    "eval_loss": 0.2348668873310089,
    "eval_Score": 47.3733851614982,
    "eval_runtime": 664.0297,
    "eval_samples_per_second": 83.7,
    "eval_steps_per_second": 4.185,
    "epoch": 20.0
}
Spanish
{
    "eval_loss": 0.6018956303596497,
    "eval_Score": 22.64911483671183,
    "eval_runtime": 352.8059,
    "eval_samples_per_second": 80.24,
    "eval_steps_per_second": 4.014,
    "epoch": 20.0
}


Step,Training Loss
500,0.272300
1000,0.272800
1500,0.270900
2000,0.271900
2500,0.271500
3000,0.271100
3500,0.270700
4000,0.269400
4500,0.269900
5000,0.270000


French


...

Step,Training Loss
500,0.232000
1000,0.233000
1500,0.231600
2000,0.232700
2500,0.232800
3000,0.232700
3500,0.232400
4000,0.231600
4500,0.232400
5000,0.232800


French


{
    "eval_loss": 0.18682506680488586,
    "eval_Score": 51.43770528400296,
    "eval_runtime": 577.2753,
    "eval_samples_per_second": 80.634,
    "eval_steps_per_second": 4.033,
    "epoch": 20.0
}
German
{
    "eval_loss": 0.21937265992164612,
    "eval_Score": 48.92384529151949,
    "eval_runtime": 664.6119,
    "eval_samples_per_second": 83.626,
    "eval_steps_per_second": 4.181,
    "epoch": 20.0
}
Spanish
{
    "eval_loss": 0.42489537596702576,
    "eval_Score": 33.85245155082943,
    "eval_runtime": 354.0141,
    "eval_samples_per_second": 79.966,
    "eval_steps_per_second": 4.0,
    "epoch": 20.0
}


Step,Training Loss
500,0.222200
1000,0.223200
1500,0.222000
2000,0.223000
2500,0.223200
3000,0.223200
3500,0.223000
4000,0.222200
4500,0.223100
5000,0.223600


Step,Training Loss
500,0.222200
1000,0.223200
1500,0.222000
2000,0.223000
2500,0.223200
3000,0.223200
3500,0.223000
4000,0.222200
4500,0.223100
5000,0.223600
